# AutoChef

## Prepare environment and process data

Data Source: https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews?resource=download

Data Source v2: https://app.roboflow.com/bens-workspace-3xdyh/fridge-detection-aymme/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true

In [2]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
import kagglehub
import pandas as pd
import numpy
import re
import PIL.Image
from ultralytics import YOLO
import torch
import cv2

C:\Users\bps78\Documents\GitHub\AutoDJ\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip install torch torchvision torchaudio

In [ ]:


processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = AutoModelForZeroShotImageClassification.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:


path = kagglehub.dataset_download("irkaal/foodcom-recipes-and-reviews")

print("Path to dataset files:", path)

In [ ]:
recipes = pd.read_csv(path + "/recipes.csv")

In [ ]:
print(recipes.shape)
display(recipes.head())
print(recipes.columns)

In [ ]:

#Convert R-style vector strings to Python lists for 'RecipeIngredientParts' column
def r_vector_to_list(s):
    # Remove c( and )
    s = s.strip()
    s = re.sub(r'^c\(|\)$', '', s)
    # Split by comma, strip quotes and whitespace
    return [item.strip().strip('"').strip("'") for item in s.split(',')]

recipes['RecipeIngredientParts'] = recipes['RecipeIngredientParts'].apply(r_vector_to_list)

## Core functionality

In [ ]:
all_ingredients = recipes['RecipeIngredientParts'].explode().unique().tolist()
print(len(all_ingredients))

Use CLIP to match images to ingredients

In [ ]:
test_image = PIL.Image.open("fridge_test.jpg")

batch_size = 100
ingredient_scores = []

for i in range(0, len(all_ingredients), batch_size):
    batch_ingredients = all_ingredients[i:i + batch_size]
    inputs = processor(text=batch_ingredients, images=test_image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    scores = outputs.logits_per_image[0].detach().cpu().numpy()
    ingredient_scores.extend(zip(batch_ingredients, scores))

In [ ]:
# Get the ingredients present in the image, sorted by score
ingredient_scores.sort(key=lambda x: x[1], reverse=True)
top_ingredients = [(ingredient, score) for ingredient, score in ingredient_scores if score > 20]
print("Top ingredients in the image:")
for ingredient, score in top_ingredients:
    print(f"{ingredient}: {score:.4f}")



Use the dataset to find recipes that match a set of ingredients

## Take 2 - use YOLO for a simplified approach

In [11]:

print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

print(torch.__version__)
print(torch.version.cuda)


CUDA Available: True
Device Name: NVIDIA GeForce GTX 1660
2.3.1+cu118
11.8
Torch: 2.3.1+cu118
Torchvision: 0.18.1+cu118
CUDA available: True
tensor([0], device='cuda:0')


In [14]:
# Load your trained model
model = YOLO('best.pt')

# Load an image
img = cv2.imread('fridge_test_3.jpeg')

# Run inference
results = model(img)

# Print detected objects
for result in results:
    boxes = result.boxes
    for box in boxes:
        class_idx = int(box.cls)
        class_name = model.names[class_idx]
        confidence = float(box.conf)
        print(f"Class: {class_name}, Confidence: {confidence:.2f}, Box: {box.xyxy}")

confidence_threshold = 0.7

for result in results:
    # Filter boxes by confidence
    high_conf_boxes = [box for box in result.boxes if float(box.conf) > confidence_threshold]
    if high_conf_boxes:
        # Optionally, update result.boxes to only include high confidence boxes
        result.boxes = high_conf_boxes
        result.show()


0: 640x480 1 chocolate, 1 corn, 13 milks, 1 potato, 1 spinach, 62.5ms
Speed: 7.3ms preprocess, 62.5ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 480)
Class: milk, Confidence: 0.91, Box: tensor([[ 888.8400,  626.5553, 1280.4944, 1124.3656]], device='cuda:0')
Class: milk, Confidence: 0.90, Box: tensor([[1907.9624,  684.7075, 2268.4905, 1140.8583]], device='cuda:0')
Class: milk, Confidence: 0.89, Box: tensor([[ 573.0219, 1892.0137,  978.7206, 2640.7859]], device='cuda:0')
Class: milk, Confidence: 0.87, Box: tensor([[ 447.3854,  870.2921,  936.7315, 1176.9474]], device='cuda:0')
Class: milk, Confidence: 0.84, Box: tensor([[ 855.5038, 1901.7666, 1362.4034, 2568.7249]], device='cuda:0')
Class: milk, Confidence: 0.81, Box: tensor([[ 262.9743, 1936.1492,  667.3904, 2673.5935]], device='cuda:0')
Class: milk, Confidence: 0.68, Box: tensor([[ 126.2663,  390.6318,  603.3668, 1074.0492]], device='cuda:0')
Class: milk, Confidence: 0.68, Box: tensor([[2400.1343, 1551.8387, 2983.3748

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="8zoDBiAT84ugEQEfOSDf")
project = rf.workspace("bens-workspace-3xdyh").project("food-item-detection-fggyf-j86bp")
version = project.version(1)
dataset = version.download("yolov8")
